## Assignment 3

Name: Annabelle Nguyen

Email: huonganh.nguyen@duke.edu

In [ ]:
# print_function for compatibility with Python 3
from __future__ import print_function

# NumPy for numerical computing
import numpy as np

# Pandas for DataFrames
import pandas as pd

# Matplotlib for visualization
from matplotlib import pyplot as plt
# display plots in the notebook
%matplotlib inline 

# Seaborn for easier visualization
import seaborn as sns

# StandardScaler from Scikit-Learn
from sklearn.preprocessing import StandardScaler

# PCA from Scikit-Learn
from sklearn.decomposition import PCA

# Scikit-Learn's KMeans algorithm
from sklearn.cluster import KMeans

In [ ]:
# load the data set
df = pd.read_csv("~/Box Sync/Duke MQM/5. Spring 2/Advanced Analytics & Applications/Assignment 3/customer_data.csv")

## Exploratory data analysis

In [ ]:
# look at the dimensions of the dataset
df.shape

In [ ]:
# look at the data types of each column in the dataset
df.dtypes

In [ ]:
# look at the top 5 rows
df.head()

In [ ]:
# descriptive statistics for numeric variables
df.describe()

In [ ]:
# descriptive statistics for categorical variables
df.describe(include=['object'])

In [ ]:
# column "Unnamed: 0" is meaningless, so it will be dropped
# column "source" contains only 1 value, so it will be dropped
df1 = df.drop(["Unnamed: 0", "source"], axis=1)
df1.head()

Level of information: product ID per order per customer (or transaction level)

In [ ]:
# Check for any missing data by variable
df1.isnull().sum()

The dataset is clean. Missing values in days_since_prior_order is acceptable because first-time customers don't have prior records of purchases.

df1 is the analytical base table.

In [ ]:
# number of unique product_id
len(df1.product_id.unique())

Now, look at some patterns in the dataset.

In [ ]:
# count the number of times each product has been bought
product_count = df1.groupby(["product_id", "product_name"]).size().sort_values(ascending = False).reset_index()
product_count.columns = ["product_id", "product_name", "number_of_items_sold"]
product_count

In [ ]:
# store the top 10 most popular products in a new dataframe
top10products = df1.groupby(["product_id", "product_name"]).size().sort_values(ascending = False).reset_index().head(10)
top10products.columns = ['product_id', 'product_name', 'number_of_items_sold']
top10products

In [ ]:
# count the number of products/items each customer has bought at the grocery store
products_per_cust = df1.groupby(["cust_id"]).agg({'product_id': [np.size]}).sort_values([('product_id','size')], ascending=False).reset_index()
products_per_cust.columns = products_per_cust.columns.map(''.join)
products_per_cust.columns = ['cust_id', 'number_of_items']
products_per_cust

In [ ]:
# count the number of times each customer has shopped at the grocery store
shopping_frequency = df1.groupby(["cust_id"]).agg({'order_id': [np.size]}).sort_values([('order_id','size')], ascending=False).reset_index()
shopping_frequency.columns = shopping_frequency.columns.map(''.join)
shopping_frequency.columns = ['cust_id', 'total_visits']
shopping_frequency

In [ ]:
# count the number of products per order
order_size = df1.groupby(["order_id"]).agg({'product_id': [np.size]}).sort_values([('product_id','size')], ascending=False).reset_index()
order_size.columns = order_size.columns.map(''.join)
order_size.columns = ['order_id', 'order_size']
order_size

In [ ]:
# aggregate total visits and total products bought by each cust_id
cust_df = df1.groupby(["cust_id"]).agg({'order_id': [np.size], 'product_id': [np.size]}).reset_index()
cust_df.columns = cust_df.columns.map(''.join)
cust_df.columns = ['cust_id', 'total_visits', 'total_products']
cust_df

## Dimensionality reduction

Test the steps for dimensionality reduction on a small sample of 2 customers.

In [ ]:
# pull out 2 random customers to create a test dataframe
test_df = df1[df1.cust_id.isin([30, 201027])]
test_df

In [ ]:
# convert the product IDs into dummy variables (test_df)
test_item_dummies = pd.get_dummies(test_df.product_id)

# Add cust_id to toy_item_dummies
test_item_dummies['cust_id'] = test_df.cust_id

# Display test_item_dummies
test_item_dummies

In [ ]:
# aggregate test_items_dummies at the customer level
test_item_data = test_item_dummies.groupby('cust_id').sum()
test_item_data

Now, to set the threshold, apply the steps for dimensionality reduction on the full data set, only pulling out the top 10 most popular products. Because the data set is too large (around 32 million rows), converting product_id into categorical variables is too computationally demanding.

In [ ]:
# get a list of product_id from top10products data set
top10productid = list(top10products.product_id)
top10productid

In [ ]:
# filter the clean data set (df1) for only the top 10 most popular products
df1_top_10_items = df1[df1.product_id.isin(top10productid)]
df1_top_10_items

In [ ]:
# look at the dimensions of df1_top_10_items
df1_top_10_items.shape

In [ ]:
# convert product_id in df1_top_10_items into dummy variables
top_10_item_data = pd.get_dummies(df1_top_10_items.product_id)

# Add cust_id to top_10_item_data
top_10_item_data['cust_id'] = df1_top_10_items.cust_id

# Display top_10_item_data
top_10_item_data

In [ ]:
# save the top_10_item_data
top_10_item_data.to_csv("~/Box Sync/Duke MQM/5. Spring 2/Advanced Analytics & Applications/Assignment 3/top_10_item_data.csv")

Now, convert product_id into categorical variable for the entire clean data set, only pulling out the top 10 frequent shoppers. Converting product_id into categorical variables for the entire data set is too computationally demanding and kills the kernel.

In [ ]:
# get a list of the top 10 cust_id from shopping_frequency data set
top10custid = list(shopping_frequency.head(10).cust_id)
top10custid

In [ ]:
# filter the clean data set (df1) for only the top 10 most frequent shoppers
df1_top_10_shoppers = df1[df1.cust_id.isin(top10custid)]
df1_top_10_shoppers

In [ ]:
# convert product_id in unique_products into dummy variables
top_10_shopper_data = pd.get_dummies(df1_top_10_shoppers.product_id)

# Add cust_id to top_10_shopper_data
top_10_shopper_data['cust_id'] = df1_top_10_shoppers.cust_id

# Display top_10_shopper_data
top_10_shopper_data

In [ ]:
# Display the dimensions of the top_10_shopper_data
top_10_shopper_data.shape

In [ ]:
# save top_10_shopper_data
top_10_shopper_data.to_csv("~/Box Sync/Duke MQM/5. Spring 2/Advanced Analytics & Applications/Assignment 3/top_10_shopper_data.csv")

## Principle Component Analysis

In [ ]:
# display the dimensions of top_10_item_data
top_10_item_data.shape

In [ ]:
# display the dimensions of top_10_shopper_data
top_10_shopper_data.shape

In [ ]:
# add an index column to top_10_shopper_data
top_10_shopper_data_index = pd.read_csv('~/Box Sync/Duke MQM/5. Spring 2/Advanced Analytics & Applications/Assignment 3/top_10_shopper_data.csv', index_col=0)

In [ ]:
# Initialize instance of StandardScaler
scaler = StandardScaler()

# Fit and transform X
X_scaled = scaler.fit_transform(top_10_shopper_data_index)

# Display first 5 rows of X_scaled
X_scaled[:5]

In [ ]:
# Fit and transform top_10_shopper_data_index
top_10_shopper_data_index_scaled = scaler.fit_transform(top_10_shopper_data_index)

# Display first 5 rows of top_10_shopper_data_index
top_10_shopper_data_index[:5]

In [ ]:
# Initialize and fit a PCA transformation
pca = PCA()
pca.fit(top_10_shopper_data_index_scaled)

In [ ]:
# Generate new features
PC_items = pca.transform(top_10_shopper_data_index_scaled)

# Display first 5 rows
PC_items[:5]

In [ ]:
# Sum of explained variance ratio
pca.explained_variance_ratio_.sum()

In [ ]:
# Cumulative explained variance
cumulative_explained_variance = np.cumsum(pca.explained_variance_ratio_)

# Plot cumulative explained variance
plt.plot(range(len(cumulative_explained_variance)), cumulative_explained_variance)

In [ ]:
# How much variance we'd capture with the first 2600 components
cumulative_explained_variance[2600]

The first 2600 components can explain about 80% of the variation.

In [ ]:
# Initialize PCA transformation, only keeping 100 components
pca = PCA(n_components=2600)

# Fit and transform top_10_shopper_data_index_scaled
PC_items = pca.fit_transform(top_10_shopper_data_index_scaled)

# Display shape of PC_items
PC_items.shape

Here, the top_10_shopper_data has been reduced from 3281 variables to 2600 variables.

In [ ]:
# Put PC_items into a dataframe
items_pca = pd.DataFrame(PC_items)

# Name the columns
items_pca.columns = ['PC{}'.format(i + 1) for i in range(PC_items.shape[1])]

# Update its index
items_pca.index = top_10_shopper_data.index

# Display first 5 rows
items_pca.head()

In [ ]:
# Save pca_item_data.csv
items_pca.to_csv('~/Box Sync/Duke MQM/5. Spring 2/Advanced Analytics & Applications/Assignment 3/pca_item_data.csv')

In [ ]:
# Save analytical base table
df1.to_csv("~/Box Sync/Duke MQM/5. Spring 2/Advanced Analytics & Applications/Assignment 3/ABT.csv")

## Clustering

In [ ]:
# Import analytical base table
base_df = pd.read_csv('~/Box Sync/Duke MQM/5. Spring 2/Advanced Analytics & Applications/Assignment 3/ABT.csv', index_col=0)

# Import thresholded item features
threshold_item_data = pd.read_csv('~/Box Sync/Duke MQM/5. Spring 2/Advanced Analytics & Applications/Assignment 3/top_10_item_data.csv', index_col=0)

# Import PCA item features
pca_item_data = pd.read_csv('~/Box Sync/Duke MQM/5. Spring 2/Advanced Analytics & Applications/Assignment 3/pca_item_data.csv', index_col=0)

In [ ]:
# Print shape of each dataframe
print( base_df.shape )
print( threshold_item_data.shape )
print( pca_item_data.shape )

In [ ]:
# Join base_df with threshold_item_data
threshold_df = base_df.merge(threshold_item_data, left_on='cust_id', right_on='cust_id', how='right')

# Display first 5 rows of threshold_df
threshold_df.head()

In [ ]:
# Join base_df with pca_item_data
pca_df = base_df.merge(pca_item_data, left_on='cust_id', right_on='cust_id', how='right')

# Display first 5 rows of pca_df
pca_df.head()

Conduct k-means on base_df

In [ ]:
# K-Means model pipeline
k_means = make_pipeline(StandardScaler(), KMeans(n_clusters=3, random_state=123))

In [ ]:
# Fit K-Means pipeline
k_means.fit(base_df)

In [ ]:
# Save clusters to base_df
base_df['cluster'] = k_means.predict(base_df)

# Display first 5 rows of base_df
base_df.head()

In [ ]:
# Scatterplot, colored by cluster
sns.lmplot(x='total_sales', y='avg_cart_value', hue='cluster', data=base_df, fit_reg=False)

Now, conduct k-means on threshold_df

In [ ]:
# K-Means model pipeline
k_means = make_pipeline(StandardScaler(), KMeans(n_clusters=3, random_state=123))

In [ ]:
# Fit K-Means pipeline
k_means.fit(threshold_df)

In [ ]:
# Save clusters to threshold_df
threshold_df['cluster'] = k_means.predict(threshold_df)

# Display first 5 rows of threshold_df
threshold_df.head()

In [ ]:
# Scatterplot, colored by cluster
sns.lmplot(x='total_sales', y='avg_cart_value', hue='cluster', data=threshold_df, fit_reg=False)

Next, conduct k-means on pca_df

In [ ]:
# K-Means model pipeline
k_means = make_pipeline(StandardScaler(), KMeans(n_clusters=3, random_state=123))

In [ ]:
# Fit K-Means pipeline
k_means.fit(pca_df)

In [ ]:
# Save clusters to pca_df
pca_df['cluster'] = k_means.predict(pca_df)

# Display first 5 rows of pca_df
pca_df.head()

In [ ]:
# Scatterplot, colored by cluster
sns.lmplot(x='total_sales', y='avg_cart_value', hue='cluster', data=pca_df, fit_reg=False)

Model comparison

In [ ]:
# Check all indices are identical
print( all( base_df.index == threshold_df.index ) )
print( all( base_df.index == pca_df.index) )

In [ ]:
# Adjusted Rand index
from sklearn.metrics import adjusted_rand_score

In [ ]:
# Similary between base_df.cluster and threshold_df.cluster
adjusted_rand_score(base_df.cluster, threshold_df.cluster)

In [ ]:
# Similary between threshold_df.cluster and base_df.cluster
adjusted_rand_score(threshold_df.cluster, pca_df.cluster)

In [ ]:
# Similary between base_df.cluster and pca_df.cluster
adjusted_rand_score(base_df.cluster, pca_df.cluster)